# Practica 4
## Captcha Destroyer hecho por: Python Haters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos cv2 (OpenCV)

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

In [ ]:
from PIL import Image
import os
import tensorflow as tf
from keras import layers, models
from keras import Model
import matplotlib.pyplot as plt #for graphs
import numpy as np
import string
import cv2


symbols = string.ascii_lowercase + string.digits # All symbols captcha can contain
directorio_input = 'samples/'
directorio_output = 'samples_transform/'
CAPTCHA_LENGTH = 5
contador = 0

#total no of images in dataset
n = len( os.listdir(directorio_input) )
imgshape = (50,200,1) #50-height, 200-width, 1-no of channels

character= string.ascii_lowercase + string.digits # All symbols captcha can contain
nchar = len(character) #total number of char possible

#preprocesss image
def preprocess():

    X = np.zeros((n,50,200,1)) #1070*50*200 array with all entries 0
    y = np.zeros((5,n,nchar)) #5*1070*36(5 letters in captcha) with all entries 0

    for i, pic in enumerate(os.listdir(directorio_input)):
    #i represents index no. of image in directory 
    #pic contains the file name of the particular image to be preprocessed at a time

        img = cv2.imread(os.path.join(directorio_input, pic), cv2.IMREAD_GRAYSCALE) #Read image in grayscale format
        pic_target = pic[:-4]#this drops the .png extension from file name and contains only the captcha for training

        if len(pic_target) < 6: #captcha is not more than 5 letters
            img = img / 255.0 #scales the image between 0 and 1
            img = np.reshape(img, (50, 200, 1)) #reshapes image to width 200 , height 50 ,channel 1 

            target=np.zeros((5,nchar)) #creates an array of size 5*36 with all entries 0

            for j, k in enumerate(pic_target):
                #j iterates from 0 to 4(5 letters in captcha)
                #k denotes the letter in captcha which is to be scanned
                index = character.find(k) #index stores the position of letter k of captcha in the character string
                target[j, index] = 1 #replaces 0 with 1 in the target array at the position of the letter in captcha

            X[i] = img #stores all the images
            y[:,i] = target #stores all the info about the letters in captcha of all images

    return X,y

#create model
def createmodel():
    img = layers.Input(shape=imgshape) # Get image as an input of size 50,200,1
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img) #50*200
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 25*100
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 13*50
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3) #to improve the stability of model
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 7*25
    
    flat = layers.Flatten()(mp3) #convert the layer into 1-D

    outs = []
    for _ in range(5): #for 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1) #drops 0.5 fraction of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res) #result of layers
    
    # Compile model and return it
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model


#Create model
model = createmodel()
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'accuracy', 'accuracy', 'accuracy', 'accuracy']
)

X, y = preprocess()

#split the 1070 samples where 970 samples will be used for training purpose
split_index = ()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]
     

#Applying the model
hist = model.fit(
    X_train, 
    [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], 
    batch_size=32, 
    epochs=60, 
    validation_split=0.2
)

#graph of loss vs epochs
for label in ["loss"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

#Loss on training set
#Finding Loss on training set
preds = model.evaluate(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]])
print ("Loss on training set= " + str(preds[0]))


#Finding loss on test set
preds = model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print ("Loss on testing set= " + str(preds[0]))

#to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    if img is not None: #image foud at file path
        img = img / 255.0 #Scale image
    else:
        print("Not detected")

    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis])) #np.newaxis=1 
    #added this bcoz x_train 970*50*200*1
    #returns array of size 1*5*36 
    result = np.reshape(res, (5, 36)) #reshape the array
    probs = []
    for i in result:
        probs.append(np.argmax(i)) #adds the index of the char found in captcha

    capt = '' #string to store predicted captcha
    for k in probs:
        capt += character[k] #finds the char corresponding to the index
    return capt 

print("Predicted Captcha =", predict(directorio_output + '2b827.png'))